### Downloading specific images
como el dataset de coco pesa muchisimo (20gb) vamos a probar 
descargar directamente las imagenes utilizando la api de pycoco para obtener los ids
y despues pegandole directamente a la url y haciendo download

De este modo descargamos solo las imagenes asociadas con las categorias especificas, que son una fraccion del dataset. 

In [5]:
from pycocotools.coco import COCO
import os
import json
import requests
from tqdm import tqdm

In [6]:
coco = COCO('annotations/instances_train2017.json')

loading annotations into memory...
Done (t=16.60s)
creating index...
index created!


In [8]:
# start with toaster since there are just a bunch of them
categories = ["toaster"]

In [9]:
subset_images_dir = 'subset_images/'
os.makedirs(subset_images_dir, exist_ok=True)

In [12]:
# Get category IDs for the specified categories
cat_ids = coco.getCatIds(catNms=categories)
print(f"Category IDs: {cat_ids}")

Category IDs: [80]


In [13]:
img_ids = coco.getImgIds(catIds=cat_ids)

In [15]:
len(img_ids)

217

In [16]:
subset_annotations = {
    'images': [],
    'annotations': [],
    'categories': coco.loadCats(cat_ids)
}

In [17]:
coco_image_base_url = "http://images.cocodataset.org/train2017/"

In [18]:
for img_id in tqdm(img_ids, desc="Processing Images"):
    img_info = coco.loadImgs(img_id)[0]
    subset_annotations['images'].append(img_info)
    
    # build image URL and download it
    img_filename = img_info['file_name']
    img_url = coco_image_base_url + img_filename
    subset_img_path = os.path.join(subset_images_dir, img_filename)
    
    # download image if not already present
    if not os.path.isfile(subset_img_path):
        response = requests.get(img_url, stream=True)
        with open(subset_img_path, 'wb') as f:
            f.write(response.content)
    
    # get annotations for this image
    ann_ids = coco.getAnnIds(imgIds=img_id, catIds=cat_ids)
    subset_annotations['annotations'].extend(coco.loadAnns(ann_ids))

with open('subset_annotations.json', 'w') as f:
    json.dump(subset_annotations, f)

print("Subset creation complete.")


Processing Images: 100%|███████████████████████████████████████████████████████████████████████| 217/217 [04:18<00:00,  1.19s/it]

Subset creation complete.
